## <center>CredCompanion</center>

This Jupyter Notebook will host all the code, calculations and data rendering that has been done to discuss the results in the PDF report. 


<a id='contents'></a>
## Table of Contents
- [Library Import](#libraries)
- [Function Definition - Define Credit Card Information](#credit_card_info)
- [Function Definition - Define Optimization Model and Decision variables](#model_definition)
- [Function Definition - Define and set Objective function](#objective_function)
- [Function Definition - Define and add Constraints](#constraints_function)
- [Function Definition - Wrapper function to create optimization model](#wrapper_function)
- [Function Definition - Export output to excel](#output_function)
- [Code Block - Import user information and extract credit card information](#user_code_block)
- [Code Block - Declare the optimization model and run it](#declare_and_run)
- [Code Block - Sensitivity Analysis](#sensitivity_analysis)
- [Code Block - Store output in excel](#store_output)

<a id='libraries'></a>
## Library Import 
[Back to table of contents](#contents)

In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import math
import tkinter as tk
from tkinter import filedialog
import datetime
from openpyxl import load_workbook

<a id='credit_card_info'></a>
## Function Definition - Define Credit Card Information
[Back to table of contents](#contents)

In [2]:
def define_credit_card_information():
    
    cred_data_categories = ['Rewards','APR Cash Advance','APR Purchases','Bonus OTP without conditions',
                        'Bonus OTP with conditions','Bonus Rewards','Income level requirement','Annual Fees']
    cred_card_data = {}
    for cat in cred_data_categories:
        cred_card_data[cat] = pd.read_excel('Credit_Card_Data.xlsx',sheet_name = cat)
    
    #Let us define the cards in variable "i"
    i = cred_card_data['Rewards']['Card'].values

    #Let us define the categories in variable"j"
    j = ['Travel','Grocery','Restaurant','Fuel','Medical','Bills','Others']

    #Let us define the number of months as specified by the user in variable "k"
    k = np.arange(1,user_num_years*12+1)
    
    #Define the Cijk matrix in a dictionary
    C = {}
    data = cred_card_data['Rewards']
    for a,card in enumerate(i):
        for b,cat in enumerate(j):
            C[str(a+1)+ '_' + str(b+1)] = data[data['Card'] == card][cat].values[0]
    
    #Define the Bonus without conditions in a dictionary
    bonus_without_cond = {}
    data = cred_card_data['Bonus OTP without conditions']
    for a,card in enumerate(i):
        bonus_without_cond[str(a+1)] = data[data['Card']==card]['Amount'].values[0]
        
    #Define the Bonus with conditions in a dictionary
    bonus_with_cond = {}
    data = cred_card_data['Bonus OTP with conditions']
    for a,card in enumerate(i):
        bonus_with_cond[str(a+1)] = data[data['Card']==card]['Amount'].values[0]
    
    #Define the APR cash advance in a dictionary
    apr_cash_advance = {}
    data = cred_card_data['APR Cash Advance']
    for a,card in enumerate(i):
        for c in range(1,user_num_years+1):
            apr_cash_advance[str(a+1) + '_' + str(c)] = data[data['Card']==card][c].values[0]
        
    #Define the APR Purchases in a dictionary
    apr_purchases = {}
    data = cred_card_data['APR Purchases']
    for a,card in enumerate(i):
        for c in range(1,user_num_years+1):
            apr_purchases[str(a+1) + '_' + str(c)] = data[data['Card']==card][c].values[0]
            
    #Define the Bonus rewards in a dictionary
    bonus_rewards = {}
    data = cred_card_data['Bonus Rewards']
    for a,card in enumerate(i):
        for b,cat in enumerate(j):
            bonus_rewards[str(a+1)+ '_' + str(b+1)] = data[data['Card'] == card][cat].values[0]
    
    #Define the income requirement in a dictionary
    income_requirement_individual = {}
    income_requirement_combined = {}
    data = cred_card_data['Income level requirement']
    for a,card in enumerate(i):
        income_requirement_individual[str(a+1)] = data[data['Card']==card]['Individual Limit'].values[0]
        income_requirement_combined[str(a+1)] = data[data['Card']==card]['Combined Household Limit'].values[0]
    for key in income_requirement_individual.keys():
        if math.isnan(income_requirement_individual[key]):
            income_requirement_individual[key] = 1000000000
    for key in income_requirement_combined.keys():
        if math.isnan(income_requirement_combined[key]):
            income_requirement_combined[key] = 1000000000
            
    #Define the Annual Fees in a dictionary
    annual_fees = {}
    data = cred_card_data['Annual Fees']
    for a,card in enumerate(i):
        for c in range(1,user_num_years+1):
            annual_fees[str(a+1) + '_' + str(c)] = data[data['Card']==card][c].values[0]
            
    return i,j,k,C,bonus_without_cond,bonus_with_cond,apr_cash_advance,apr_purchases,bonus_rewards,income_requirement_individual, income_requirement_combined,annual_fees

<a id='model_definition'></a>
## Function Definition - Define Optimization Model and Decision variables
[Back to table of contents](#contents)

In [3]:
#Define and model and create the variables

def credcompanion_define(model_name,i,j,k,user_payments,user_loans):
    # Create a new model
    model_cred_companion = gp.Model(model_name)
    
    #Adding the Xijk variable for spending amounts
    X = {}
    for a in range(1,len(i)+1):
        for b in range(1,len(j)+1):
            for c in k:
                n = 'X_' + str(a) + '_' + str(b) + '_' + str(c)
                X[str(a) + '_' + str(b) + '_' + str(c)] = model_cred_companion.addVar(vtype=GRB.CONTINUOUS, name= n)
    model_cred_companion.update()
    
    #Adding the Yi variable for card selection
    Y = {}
    for a in range(1,len(i)+1):
        n = 'Y_' + str(a)
        Y[str(a)] = model_cred_companion.addVar(vtype=GRB.BINARY, name= n)
    model_cred_companion.update()
    
    #Adding the Y_APR_Purchase_i variable for card selection for APR purchases
    Y_APR_purchase = {}
    if user_payments == 'No':
        for a in range(1,len(i)+1):
            n = 'Y_APR_purchase_' + str(a)
            Y_APR_purchase[str(a)] = model_cred_companion.addVar(vtype=GRB.BINARY, name= n)
        model_cred_companion.update()
    
    #Adding the X_APR_Purchase_i variable for amounts for APR purchases in each card
    X_APR_purchase = {}
    if user_payments == 'No':
        for a in range(1,len(i)+1):
            for c in range(1,int(len(k)/12+1)):
                n = 'X_APR_purchase_' + str(a) + '_' + str(c)
                X_APR_purchase[str(a) + '_' + str(c)] = model_cred_companion.addVar(vtype=GRB.CONTINUOUS, name= n)
        model_cred_companion.update()
    
    #Adding the Y_APR_cash_i variable for card selection for APR cash advance
    Y_APR_cash = {}
    if user_loans == 'Yes':
        for a in range(1,len(i)+1):
            n = 'Y_APR_cash_' + str(a)
            Y_APR_cash[str(a)] = model_cred_companion.addVar(vtype=GRB.BINARY, name= n)
        model_cred_companion.update()
        
    #Adding the Zi variable for bonus inclusion with conditions
    Z = {}
    for a in range(1,len(i)+1):
        n = 'Z_' + str(a)
        Z[str(a)] = model_cred_companion.addVar(vtype=GRB.BINARY, name= n)

    #Adding Zi variable for additional bonus
    a = 11
    n = 'Z_' + str(a) + '_addnl'
    Z[str(a) + '_addnl'] = model_cred_companion.addVar(vtype=GRB.BINARY, name= n)    

    model_cred_companion.update()
    
    #Adding the Pi variable for individual income
    P = {}
    for a in range(1,len(i)+1):
        n = 'P_' + str(a)
        P[str(a)] = model_cred_companion.addVar(vtype=GRB.BINARY, name= n)
    model_cred_companion.update()
    
    #Adding the Qi variable for combined household income
    Q = {}
    for a in range(1,len(i)+1):
        n = 'Q_' + str(a)
        Q[str(a)] = model_cred_companion.addVar(vtype=GRB.BINARY, name= n)
    model_cred_companion.update()
    
    return model_cred_companion, X, Y, Y_APR_purchase,X_APR_purchase, Y_APR_cash, Z, P, Q

<a id='objective_function'></a>
## Function Definition - Define and set Objective function
[Back to table of contents](#contents)

In [4]:
#Set the objective

def credcompanion_setobjective(model_cred_companion,i,j,k,X,Y,Y_APR_purchase,X_APR_purchase,Y_APR_cash,Z,bonus_without_cond,
                               bonus_with_cond,annual_fees,user_loans_amount,user_loans_months,apr_cash_advance,
                               user_payments_amount,user_payments_months,apr_purchases):
    #Add rewards from spending in objective
    obj = 0
    for a in range(1,len(i)+1):
        for b in range(1,len(j)+1):
            for c in k:
                obj = obj + X[str(a) + '_' + str(b) + '_' + str(c)] * C[str(a)+ '_' + str(b)] * Y[str(a)]
                
    #Add one time bonus without conditions in objective
    for a in range(1,len(i)+1):
        obj = obj + bonus_without_cond[str(a)]*Y[str(a)]
        
    #Add one time bonus with conditions in objective
    for a in range(1,len(i)+1):
        obj = obj + bonus_with_cond[str(a)]*Y[str(a)]*Z[str(a)]

    a = 11
    obj = obj + 100*Y[str(a)]*Z[str(a) + '_addnl']
    
    #Add yearly fees in objective
    for a in range(1,len(i)+1):
        for c in range(1,user_num_years+1):
            obj = obj - annual_fees[str(a) + '_' + str(c)]*Y[str(a)]
            
    #Add yearly APR cash advance fees
    if user_loans == 'Yes':
        for a in range(1,len(i)+1):
            for c in range(1,user_num_years+1):
                obj = obj - user_loans_amount * (pow((1 + apr_cash_advance[str(a) + '_' + str(c)]/1200),user_loans_months) - 1) * Y_APR_cash[str(a)]
        
    #Add yearly APR purchases fees
    if user_payments == 'No':
        for a in range(1,len(i)+1):
            for c in range(1,user_num_years+1):
                obj = obj - X_APR_purchase[str(a) + '_' + str(c)] * (pow((1 + apr_purchases[str(a) + '_' + str(c)]/1200),user_payments_months) - 1) * Y_APR_purchase[str(a)]
    
    model_cred_companion.setObjective(obj,GRB.MAXIMIZE)
    model_cred_companion.update()
    
    return model_cred_companion

<a id='constraints_function'></a>
## Function Definition - Define and add Constraints
[Back to table of contents](#contents)

In [5]:
#Add the constraints

def credcompanion_setconstraints(model_cred_companion,i,j,k,X,Y,Y_APR_purchase,X_APR_purchase,Y_APR_cash,Z,P, Q,
                                 user_num_cards,user_loans,user_payments,user_payments_amount,
                                 user_grocery_min,user_grocery_max,
                                 user_restaurant_min,user_restaurant_max,
                                 user_bills_min,user_bills_max,
                                 user_fuel_min,user_fuel_max,
                                 user_travel_min, user_travel_max,
                                 user_drugs_min, user_drugs_max,
                                 user_others_min, user_others_max,
                                 income_requirement_individual, income_requirement_combined,
                                ):
    
    #Adding the Yi constraint
    y_constraint = 0
    for key in Y.keys():
        y_constraint = y_constraint + Y[key]
    model_cred_companion.addConstr(y_constraint <= user_num_cards, "number_of_cards_constraint")
    model_cred_companion.update()
    
    #Adding constraint for APR cash advance
    if user_loans == 'Yes':
        y_apr_cash_constraint = 0
        for key in Y_APR_cash.keys():
            y_apr_cash_constraint = y_apr_cash_constraint + Y_APR_cash[key]
        model_cred_companion.addConstr(y_apr_cash_constraint == 1, "apr_cash_constraint")
        model_cred_companion.update()
        
    #Adding constraint for APR cash advance and Yi relationship
    if user_loans == 'Yes':
        for key in Y_APR_cash.keys():
            model_cred_companion.addConstr(Y_APR_cash[key] <= Y[key], "Y_APR_Cash_" + str(key))
            model_cred_companion.update()
            
    #Adding constraint for APR purchase
    if user_payments == 'No':
        for c in range(1,user_num_years+1):
            x_apr_purchase_constraint = 0
            for a in range(1,len(i)+1):
                x_apr_purchase_constraint = x_apr_purchase_constraint + X_APR_purchase[str(a) + '_' + str(c)] * Y_APR_purchase[str(a)]
            model_cred_companion.addConstr(x_apr_purchase_constraint == user_payments_amount, "APR_purchase_constraint_" + str(a) + "_" + str(c))
        model_cred_companion.update()
        
    #Adding constraint for APR purchase amounts
    if user_payments == 'No':
        for a in range(1,len(i)+1):
            for temp in range(0,user_num_years):
                purchase_constraint = 0
                for c in range(temp*12+1,temp*12+13):
                    for b in range(1,len(j)+1):
                        purchase_constraint = purchase_constraint + X[str(a) + '_' + str(b) + '_' + str(c)]
                model_cred_companion.addConstr(purchase_constraint >= X_APR_purchase[str(a) + '_' + str(temp+1)] * Y_APR_purchase[str(a)] , "Y_APR_Purchase_Amount_Constraint_" + str(a) + '_' + str(temp+1))
        model_cred_companion.update()
        
    #Adding constraint for APR purchases and Yi relationship
    if user_payments == 'No':
        for a in range(1,len(i)+1):
            model_cred_companion.addConstr(Y_APR_purchase[key] <= Y[key], "Y_APR_Purchase_" + str(a))
            model_cred_companion.update()
            
    #Adding constraint for APR purchases cards and purchase amount relationship
    if user_payments == 'No':
        for a in range(1,len(i)+1):
            for c in range(1,user_num_years+1):
                model_cred_companion.addConstr(Y_APR_purchase[str(a)] <= X_APR_purchase[str(a) + '_' + str(c)], "X_Y_APR_Purchase_" + str(a) + "_" + str(c))
            model_cred_companion.update()
            
    #Adding the constraints for various items (min and max range constraints)
    #Adding groceries constraint
    b = j.index('Grocery') + 1
    for c in k:
        grocery_constraint = 0
        for a in range(1,len(i)+1):
            grocery_constraint = grocery_constraint + X[str(a) + '_' + str(b) + '_' + str(c)] * Y[str(a)]
        model_cred_companion.addConstr(grocery_constraint >= user_grocery_min, "grocery_constraint_min_" + str(c))
        model_cred_companion.addConstr(grocery_constraint <= user_grocery_max, "grocery_constraint_max_" + str(c))
    model_cred_companion.update()
    #Adding restaurant constraint
    b = j.index('Restaurant') + 1
    for c in k:
        restaurant_constraint = 0
        for a in range(1,len(i)+1):
            restaurant_constraint = restaurant_constraint + X[str(a) + '_' + str(b) + '_' + str(c)] * Y[str(a)]
        model_cred_companion.addConstr(restaurant_constraint >= user_restaurant_min, "restaurant_constraint_min_" + str(c))
        model_cred_companion.addConstr(restaurant_constraint <= user_restaurant_max, "restaurant_constraint_max_" + str(c))
    model_cred_companion.update()
    #Adding Bills constraint
    b = j.index('Bills') + 1
    for c in k:
        bills_constraint = 0
        for a in range(1,len(i)+1):
            bills_constraint = bills_constraint + X[str(a) + '_' + str(b) + '_' + str(c)] * Y[str(a)]
        model_cred_companion.addConstr(bills_constraint >= user_bills_min, "bills_constraint_min_" + str(c))
        model_cred_companion.addConstr(bills_constraint <= user_bills_max, "bills_constraint_max_" + str(c))
    model_cred_companion.update()
    #Adding fuel constraint
    b = j.index('Fuel') + 1
    for c in k:
        fuel_constraint = 0
        for a in range(1,len(i)+1):
            fuel_constraint = fuel_constraint + X[str(a) + '_' + str(b) + '_' + str(c)] * Y[str(a)]
        model_cred_companion.addConstr(fuel_constraint >= user_fuel_min, "fuel_constraint_min_" + str(c))
        model_cred_companion.addConstr(fuel_constraint <= user_fuel_max, "fuel_constraint_max_" + str(c))
    model_cred_companion.update()
    #Adding Travel constraint
    b = j.index('Travel') + 1
    for c in k:
        travel_constraint = 0
        for a in range(1,len(i)+1):
            travel_constraint = travel_constraint + X[str(a) + '_' + str(b) + '_' + str(c)] * Y[str(a)]
        model_cred_companion.addConstr(travel_constraint >= user_travel_min, "travel_constraint_min_" + str(c))
        model_cred_companion.addConstr(travel_constraint <= user_travel_max, "travel_constraint_max_" + str(c))
    model_cred_companion.update()
    #Adding Medical constraint
    b = j.index('Medical') + 1
    for c in k:
        medical_constraint = 0
        for a in range(1,len(i)+1):
            medical_constraint = medical_constraint + X[str(a) + '_' + str(b) + '_' + str(c)] * Y[str(a)]
        model_cred_companion.addConstr(medical_constraint >= user_drugs_min, "medical_constraint_min_" + str(c))
        model_cred_companion.addConstr(medical_constraint <= user_drugs_max, "medical_constraint_max_" + str(c))
    model_cred_companion.update()
    #Adding Other constraint
    b = j.index('Others') + 1
    for c in k:
        other_constraint = 0
        for a in range(1,len(i)+1):
            other_constraint = other_constraint + X[str(a) + '_' + str(b) + '_' + str(c)] * Y[str(a)]
        model_cred_companion.addConstr(other_constraint >= user_others_min, "other_constraint_min_" + str(c))
        model_cred_companion.addConstr(other_constraint <= user_others_max, "other_constraint_max_" + str(c))
    model_cred_companion.update()
    
    #Adding income level constraint
    for a in range(1,len(i)+1):
        model_cred_companion.addConstr(user_income_individual >= income_requirement_individual[str(a)]*P[str(a)], "indi_income_constraint_" + str(a))
    for a in range(1,len(i)+1):
        model_cred_companion.addConstr(user_income_house >= income_requirement_combined[str(a)]*Q[str(a)], "comb_income_constraint_" + str(a))
    for a in range(1,len(i)+1):
        model_cred_companion.addConstr(Y[str(a)] <= P[str(a)]  + Q[str(a)], "overall_income_constraint_" + str(a))
    model_cred_companion.update()
    
    #Adding constraint to limit X to 0 if card not chosen
    for a in range(1,len(i)+1):
        for b in range(1,len(j)+1):
            for c in k:
                model_cred_companion.addConstr(X[str(a) + '_' + str(b) + '_' + str(c)] <= Y[str(a)] * 1000000000,"X_limiting_constraint_" + str(a) + '_' + str(b) + '_' + str(c))
    model_cred_companion.update()
    
    #Adding constraint for one time bonus with conditions

    #TD® Aeroplan® Visa Infinite Privilege* Credit Card
    a = 6
    spending_amount = 0
    for b in range(1,len(j)+1):
        for c in range(1,4):
            spending_amount = spending_amount + X[str(a) + '_' + str(b) + '_' + str(c)] * Y[str(a)]
    model_cred_companion.addConstr(spending_amount >= 3000 * Z[str(a)], "bonus_with_condn_" + str(a))


    #TD First Class Travel®Visa Infinite* Card
    a = 7
    spending_amount = 0
    for b in range(1,len(j)+1):
        for c in range(1,4):
            spending_amount = spending_amount + X[str(a) + '_' + str(b) + '_' + str(c)] * Y[str(a)]
    model_cred_companion.addConstr(spending_amount >= 1500 * Z[str(a)], "bonus_with_condn_" + str(a))


    #TD Platinum Travel Visa* Card
    a = 8
    spending_amount = 0
    for b in range(1,len(j)+1):
        for c in range(1,4):
            spending_amount = spending_amount + X[str(a) + '_' + str(b) + '_' + str(c)] * Y[str(a)]
    model_cred_companion.addConstr(spending_amount >= 1000 * Z[str(a)], "bonus_with_condn_" + str(a))


    #Scotiabank Passport™ Visa Infinite* Card
    a = 11
    spending_amount = 0
    for b in range(1,len(j)+1):
        for c in range(1,4):
            spending_amount = spending_amount + X[str(a) + '_' + str(b) + '_' + str(c)] * Y[str(a)]
    model_cred_companion.addConstr(spending_amount >= 1000 * Z[str(a)], "bonus_with_condn_" + str(a))

    #Scotiabank Passport™ Visa Infinite* Card (Additional bonus)
    a = 11
    spending_amount = 0
    for b in range(1,len(j)+1):
        for c in range(1,13):
            spending_amount = spending_amount + X[str(a) + '_' + str(b) + '_' + str(c)] * Y[str(a)]
    model_cred_companion.addConstr(spending_amount >= 40000 * Z[str(a) + '_addnl'], "bonus_with_condn_" + str(a) + '_addnl')


    #Scotiabank Gold American Express® Card
    a = 12
    spending_amount = 0
    for b in range(1,len(j)+1):
        for c in range(1,4):
            spending_amount = spending_amount + X[str(a) + '_' + str(b) + '_' + str(c)] * Y[str(a)]
    model_cred_companion.addConstr(spending_amount >= 1000 * Z[str(a)], "bonus_with_condn_" + str(a))


    #CIBC Aventura® Visa Infinite* Card
    a = 25
    spending_amount = 0
    for b in range(1,len(j)+1):
        for c in range(1,5):
            spending_amount = spending_amount + X[str(a) + '_' + str(b) + '_' + str(c)] * Y[str(a)]
    model_cred_companion.addConstr(spending_amount >= 3000 * Z[str(a)], "bonus_with_condn_" + str(a))


    #CIBC Aventura® Visa Infinite Privilege* Card
    a = 29
    spending_amount = 0
    for b in range(1,len(j)+1):
        for c in range(1,5):
            spending_amount = spending_amount + X[str(a) + '_' + str(b) + '_' + str(c)] * Y[str(a)]
    model_cred_companion.addConstr(spending_amount >= 3000 * Z[str(a)], "bonus_with_condn_" + str(a))


    #CIBC AEROPLAN® VISA INFINITE PRIVILEGE* CARD
    a = 30
    spending_amount = 0
    for b in range(1,len(j)+1):
        for c in k:
            spending_amount = spending_amount + X[str(a) + '_' + str(b) + '_' + str(c)] * Y[str(a)]
    model_cred_companion.addConstr(spending_amount >= 25000 * Z[str(a)], "bonus_with_condn_" + str(a))


    #CIBC Aventura® Gold Visa* Card
    a = 31
    spending_amount = 0
    for b in range(1,len(j)+1):
        for c in range(1,5):
            spending_amount = spending_amount + X[str(a) + '_' + str(b) + '_' + str(c)] * Y[str(a)]
    model_cred_companion.addConstr(spending_amount >= 3000 * Z[str(a)], "bonus_with_condn_" + str(a))
    model_cred_companion.update()
    
    return model_cred_companion

<a id='wrapper_function'></a>
## Function Definition - Wrapper function to create optimization model
[Back to table of contents](#contents)

In [6]:
def credcompanion_calculate(modelname,i,j,k,C,bonus_without_cond,bonus_with_cond,apr_cash_advance,apr_purchases,
                            bonus_rewards,annual_fees,
                            user_num_cards,user_loans,user_loans_amount,user_loans_months,user_payments,user_payments_amount,user_payments_months,
                            user_grocery_min,user_grocery_max,
                            user_restaurant_min,user_restaurant_max,
                            user_bills_min,user_bills_max,
                            user_fuel_min,user_fuel_max,
                            user_travel_min, user_travel_max,
                            user_drugs_min, user_drugs_max,
                            user_others_min, user_others_max,
                            income_requirement_individual, income_requirement_combined):
    #Define model
    model_cred_companion, X, Y, Y_APR_purchase,X_APR_purchase, Y_APR_cash, Z, P, Q = credcompanion_define(modelname,i,j,k,user_payments,user_loans)
    #Define objective
    model_cred_companion = credcompanion_setobjective(model_cred_companion,i,j,k,X,Y,Y_APR_purchase,X_APR_purchase,Y_APR_cash,Z,bonus_without_cond,
                               bonus_with_cond,annual_fees,user_loans_amount,user_loans_months,apr_cash_advance,
                               user_payments_amount,user_payments_months,apr_purchases)
    #Define constraints
    model_cred_companion = credcompanion_setconstraints(model_cred_companion,i,j,k,X,Y,Y_APR_purchase,X_APR_purchase,Y_APR_cash,Z,P, Q,
                                 user_num_cards,user_loans,user_payments,user_payments_amount,
                                 user_grocery_min,user_grocery_max,
                                 user_restaurant_min,user_restaurant_max,
                                 user_bills_min,user_bills_max,
                                 user_fuel_min,user_fuel_max,
                                 user_travel_min, user_travel_max,
                                 user_drugs_min, user_drugs_max,
                                 user_others_min, user_others_max,
                                 income_requirement_individual, income_requirement_combined
                                )
    return model_cred_companion,X,Y,Z,Y_APR_cash,Y_APR_purchase,X_APR_purchase

<a id='output_function'></a>
## Function Definition - Export Output to excel
[Back to table of contents](#contents)

In [7]:
def credcompanion_output(file_path,model_cred_companion,i,j,k,X,Y,C,Z,bonus_without_cond,bonus_with_cond,user_num_years,
                         annual_fees,user_loans,user_payments,user_loans_amount,apr_cash_advance,user_loans_months,Y_APR_cash,
                        Y_APR_purchase,X_APR_purchase,apr_purchases,user_payments_months,
                        travel_statement,grocery_statement,restaurant_statement,fuel_statement,drugs_statement,
                     bills_statement,others_statement,loans_statement,payments_statement):
        
    temp = file_path.split('/')
    temp[len(temp)-1].split('.')[0]
    name = temp[len(temp)-1].split('.')[0] + "{:%Y_%b_%d_%H_%M_%S}".format(datetime.datetime.now()) + '.xlsx'
    
    #COVER PAGE
    
    outputcolumns = ['Description','Value']
    df_coverpage = pd.DataFrame(columns=outputcolumns)
    
    #Selected cards
    new_dict = {}
    new_dict['Description'] = "Selected Cards"
    temp = []
    for a in range(1,len(i)+1):
        if Y[str(a)].x != 0:
            temp.append(i[a-1])
    new_dict['Value'] = temp
    df_coverpage = df_coverpage.append(new_dict, ignore_index=True)
    
    #Total Rewards
    new_dict = {}
    new_dict['Description'] = "Total Rewards"
    temp = 0
    for a in range(1,len(i)+1):
        for b in range(1,len(j)+1):
            for c in k:
                temp = temp + X[str(a) + '_' + str(b) + '_' + str(c)].x * C[str(a)+ '_' + str(b)] * Y[str(a)].x
    new_dict['Value'] = temp
    df_coverpage = df_coverpage.append(new_dict, ignore_index=True)
    
    #Total Bonus
    new_dict = {}
    new_dict['Description'] = "Total Bonus"
    temp = 0
    #Add one time bonus without conditions
    for a in range(1,len(i)+1):
        temp = temp + bonus_without_cond[str(a)] * Y[str(a)].x
    #Add one time bonus with conditions
    for a in range(1,len(i)+1):
        temp = temp + bonus_with_cond[str(a)] * Y[str(a)].x * Z[str(a)].x
    new_dict['Value'] = temp
    df_coverpage = df_coverpage.append(new_dict, ignore_index=True)
    
    #Total Annual Fees
    new_dict = {}
    new_dict['Description'] = "Total annual Fees on Selected Cards"
    temp = 0
    for a in range(1,len(i)+1):
        for c in range(1,user_num_years+1):
            temp = temp + annual_fees[str(a) + '_' + str(c)] * Y[str(a)].x
    new_dict['Value'] = temp
    df_coverpage = df_coverpage.append(new_dict, ignore_index=True)
    
    #Selected card and value for APR cash advance
    if user_loans == 'Yes':
        new_dict = {}
        new_dict['Description'] = "Selected card for Cash Advance"
        temp = []
        for a in range(1,len(i)+1):
            if Y_APR_cash[str(a)].x != 0:
                temp.append(i[a-1])
        new_dict['Value'] = temp
        df_coverpage = df_coverpage.append(new_dict, ignore_index=True)
        new_dict = {}
        new_dict['Description'] = "Total Cash Advance Interest"
        temp = 0
        for a in range(1,len(i)+1):
            for c in range(1,user_num_years+1):
                temp = temp + user_loans_amount * (pow((1 + apr_cash_advance[str(a) + '_' + str(c)]/1200),user_loans_months) - 1) * Y_APR_cash[str(a)].x
        new_dict['Value'] = temp
        df_coverpage = df_coverpage.append(new_dict, ignore_index=True)
    
    #Selected card for APR purchases
    if user_payments == 'No':
        new_dict = {}
        new_dict['Description'] = "Selected card for Purchase Repayment Shortfalls"
        temp = []
        for a in range(1,len(i)+1):
            if Y_APR_purchase[str(a)].x != 0:
                temp.append(i[a-1])
        new_dict['Value'] = temp
        df_coverpage = df_coverpage.append(new_dict, ignore_index=True)
        new_dict = {}
        new_dict['Description'] = "Total Purchase Repayment Interest"
        temp = 0
        for a in range(1,len(i)+1):
            for c in range(1,user_num_years+1):
                temp = temp + X_APR_purchase[str(a) + '_' + str(c)].x * (pow((1 + apr_purchases[str(a) + '_' + str(c)]/1200),user_payments_months) - 1) * Y_APR_purchase[str(a)].x
        new_dict['Value'] = temp
        df_coverpage = df_coverpage.append(new_dict, ignore_index=True)
    
    
    #Total Overall Benefits - Cost
    new_dict = {}
    new_dict['Description'] = "Total Overall Benefit - Cost"
    new_dict['Value'] = model_cred_companion.objVal
    df_coverpage = df_coverpage.append(new_dict, ignore_index=True)
    
    df_coverpage.to_excel(name,sheet_name = "Results",index=False)
    
    
    
    book = load_workbook(name)
    writer = pd.ExcelWriter(name, engine='openpyxl') 
    writer.book = book
    
    
    #SPENDING AMOUNTS PAGE
    
    outputcolumns = ['Card','Category']
    for c in k:
        outputcolumns.append('Month ' + str(c))
    df_monthly_spending = pd.DataFrame(columns=outputcolumns)
    for a in range(1,len(i)+1):
        if Y[str(a)].x != 0:
            for b in range(1,len(j)+1):
                new_dict = {}
                new_dict['Card'] = i[a-1]
                new_dict['Category'] = j[b-1]
                for c in k:
                    new_dict['Month ' + str(c)] = X[str(a) + '_' + str(b) + '_' + str(c)].x
                df_monthly_spending = df_monthly_spending.append(new_dict, ignore_index=True)
    
    df_monthly_spending.to_excel(writer,sheet_name = "Spending_Amounts",index=False)
    writer.save()
    
    #APR PURCHASES AMOUNTS PAGE
    if user_payments == 'No':
        outputcolumns = ['Card']
        for c in range(1,user_num_years+1):
            outputcolumns.append('Year ' + str(c))
        df_apr_purchases = pd.DataFrame(columns=outputcolumns)
        for a in range(1,len(i)+1):
            if Y_APR_purchase[str(a)].x != 0:
                new_dict = {}
                new_dict['Card'] = i[a-1]
                for c in range(1,user_num_years+1):
                    new_dict['Year ' + str(c)] = X_APR_purchase[str(a) + '_' + str(c)].x
                df_apr_purchases = df_apr_purchases.append(new_dict, ignore_index=True)
        df_apr_purchases.to_excel(writer,sheet_name = "Purchase_Shortfall_Amounts",index=False)
        writer.save()
        
        
    #Sensitivity Analysis page
    outputcolumns = ['Category','Sensitivity Analysis']
    df_sensitivity = pd.DataFrame(columns=outputcolumns)
    df_sensitivity = df_sensitivity.append({'Category':'Travel','Sensitivity Analysis':travel_statement}, ignore_index=True)
    df_sensitivity = df_sensitivity.append({'Category':'Groceries','Sensitivity Analysis':grocery_statement}, ignore_index=True)
    df_sensitivity = df_sensitivity.append({'Category':'Restaurants','Sensitivity Analysis':restaurant_statement}, ignore_index=True)
    df_sensitivity = df_sensitivity.append({'Category':'Fuel','Sensitivity Analysis':fuel_statement}, ignore_index=True)
    df_sensitivity = df_sensitivity.append({'Category':'Drugs','Sensitivity Analysis':drugs_statement}, ignore_index=True)
    df_sensitivity = df_sensitivity.append({'Category':'Bills','Sensitivity Analysis':bills_statement}, ignore_index=True)
    df_sensitivity = df_sensitivity.append({'Category':'Others','Sensitivity Analysis':others_statement}, ignore_index=True)
    if user_loans == 'Yes':
        df_sensitivity = df_sensitivity.append({'Category':'Cash Advance','Sensitivity Analysis':loans_statement}, ignore_index=True)
    if user_payments == 'No':
        df_sensitivity = df_sensitivity.append({'Category':'Credit Repayment Shortfall','Sensitivity Analysis':payments_statement}, ignore_index=True)
    df_sensitivity.to_excel(writer,sheet_name = "Sensitivity_Analysis",index=False)
    writer.save()

<a id='user_code_block'></a>
## Code Block - Import user information and extract credit card information
[Back to table of contents](#contents)

In [8]:
root = tk.Tk()
root.withdraw()

file_path = filedialog.askopenfilename()

We will extract the user information and assign variables to them

In [9]:
user_inputs = pd.read_excel(file_path)

user_inputs['S.No'] = user_inputs['S.No'].astype(str)

user_num_years = user_inputs[user_inputs['S.No'] == '1']['Response'].values[0]

user_num_cards = user_inputs[user_inputs['S.No'] == '2']['Response'].values[0]

user_travel_min = user_inputs[user_inputs['S.No'] == '3.a']['Response'].values[0]
user_travel_max = user_inputs[user_inputs['S.No'] == '3.a']['Unnamed: 3'].values[0]

user_grocery_min = user_inputs[user_inputs['S.No'] == '3.b']['Response'].values[0]
user_grocery_max = user_inputs[user_inputs['S.No'] == '3.b']['Unnamed: 3'].values[0]

user_restaurant_min = user_inputs[user_inputs['S.No'] == '3.c']['Response'].values[0]
user_restaurant_max = user_inputs[user_inputs['S.No'] == '3.c']['Unnamed: 3'].values[0]

user_fuel_min = user_inputs[user_inputs['S.No'] == '3.d']['Response'].values[0]
user_fuel_max = user_inputs[user_inputs['S.No'] == '3.d']['Unnamed: 3'].values[0]

user_drugs_min = user_inputs[user_inputs['S.No'] == '3.e']['Response'].values[0]
user_drugs_max = user_inputs[user_inputs['S.No'] == '3.e']['Unnamed: 3'].values[0]

user_bills_min = user_inputs[user_inputs['S.No'] == '3.f']['Response'].values[0]
user_bills_max = user_inputs[user_inputs['S.No'] == '3.f']['Unnamed: 3'].values[0]

user_others_min = user_inputs[user_inputs['S.No'] == '3.g']['Response'].values[0]
user_others_max = user_inputs[user_inputs['S.No'] == '3.g']['Unnamed: 3'].values[0]

user_payments = user_inputs[user_inputs['S.No'] == '4.a']['Response'].values[0]

user_payments_amount = user_inputs[user_inputs['S.No'] == '4.b']['Response'].values[0]

user_payments_months = user_inputs[user_inputs['S.No'] == '4.c']['Response'].values[0]

user_loans = user_inputs[user_inputs['S.No'] == '5.a']['Response'].values[0]

user_loans_amount = user_inputs[user_inputs['S.No'] == '5.b']['Response'].values[0]

user_loans_months = user_inputs[user_inputs['S.No'] == '5.c']['Response'].values[0]

user_income_individual = user_inputs[user_inputs['S.No'] == '6.a']['Response'].values[0]

user_income_house = user_inputs[user_inputs['S.No'] == '6.b']['Response'].values[0]

Next, we will import the credit card information sheet

In [10]:
i,j,k,C,bonus_without_cond,bonus_with_cond,apr_cash_advance,apr_purchases,bonus_rewards,income_requirement_individual, income_requirement_combined,annual_fees = define_credit_card_information()

In [11]:
for a,b in enumerate(j):
    print(a+1,b)

1 Travel
2 Grocery
3 Restaurant
4 Fuel
5 Medical
6 Bills
7 Others


In [12]:
for a,b in enumerate(i):
    print(a+1,b)

1 TD® Aeroplan® Visa Infinite* Card
2 TD® Aeroplan® Visa Platinum* Card
3 TD Rewards Visa* Card
4 TD Cash Back Visa Infinite* Card
5 TD Cash Back Visa* Card
6 TD® Aeroplan® Visa Infinite Privilege* Credit Card
7 TD First Class Travel®Visa Infinite* Card
8 TD Platinum Travel Visa* Card
9 TD Emerald Flex Rate Visa Card
10 Scotia Momentum® Visa Infinite* Card
11 Scotiabank Passport™ Visa Infinite* Card
12 Scotiabank Gold American Express® Card
13 Scotia Momentum® Mastercard®* Credit Card
14 RBC Avion Visa Infinite
15 RBC Rewards+ Visa
16 RBC Cash Back Preferred World Elite Mastercard
17 RBC Cash Back Mastercard
18 RBC Visa Classic Low Rate Option
19 RBC Avion Visa Infinite Privilege
20 Signature RBC Rewards Visa
21 RBC Avion Visa Platinum
22 RBC Visa Platinum
23 RBC Rewards Visa Preferred
24 RBC RateAdvantage Visa
25 CIBC Aventura® Visa Infinite* Card
26 CIBC AEROPLAN® VISA INFINITE* CARD
27 CIBC Dividend® Visa Infinite* Card
28 CIBC Classic Visa* Card
29 CIBC Aventura® Visa Infinite Priv

<a id='declare_and_run'></a>
## Code Block - Declare the optimization model and run it
[Back to table of contents](#contents)

We will now define the model and set the objective and constraints

In [13]:
model_cred_companion,X,Y,Z,Y_APR_cash,Y_APR_purchase,X_APR_purchase = credcompanion_calculate('Cred_Companion',i,j,k,C,bonus_without_cond,bonus_with_cond,apr_cash_advance,apr_purchases,
                            bonus_rewards,annual_fees,
                            user_num_cards,user_loans,user_loans_amount,user_loans_months,user_payments,user_payments_amount,user_payments_months,
                            user_grocery_min,user_grocery_max,
                            user_restaurant_min,user_restaurant_max,
                            user_bills_min,user_bills_max,
                            user_fuel_min,user_fuel_max,
                            user_travel_min, user_travel_max,
                            user_drugs_min, user_drugs_max,
                            user_others_min, user_others_max,
                            income_requirement_individual, income_requirement_combined)

Using license file C:\Users\Manthan\gurobi.lic
Academic license - for non-commercial use only - expires 2021-01-07


In [14]:
model_cred_companion.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0xce0c9083
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 0.15s
Presolved: 44785 rows, 58733 columns, 142540 nonzeros
Presolved model has 29312 SOS constraint(s)
Variable types: 43968 continuous, 14765 integer (14765 binary)
Found

<a id='sensitivity_analysis'></a>
## Code Block - Sensitivity Analysis
[Back to table of contents](#contents)

In [15]:
#Define the range for travel
if user_travel_min != user_travel_max:
    travel_spending_delta = int((user_travel_max - user_travel_min)*5/100)
    diff = 0
    new_max = 0
    for delta in range(user_travel_max,user_travel_min,-int(travel_spending_delta)):
        model_cred_companion_new,X_new,Y_new,Z_new,Y_APR_cash_new,Y_APR_purchase_new,X_APR_purchase_new = credcompanion_calculate('Cred_Companion_Travel_' + str(delta),i,j,k,C,bonus_without_cond,bonus_with_cond,apr_cash_advance,apr_purchases,
                                bonus_rewards,annual_fees,
                                user_num_cards,user_loans,user_loans_amount,user_loans_months,user_payments,user_payments_amount,user_payments_months,
                                user_grocery_min,user_grocery_max,
                                user_restaurant_min,user_restaurant_max,
                                user_bills_min,user_bills_max,
                                user_fuel_min,user_fuel_max,
                                user_travel_min, delta,
                                user_drugs_min, user_drugs_max,
                                user_others_min, user_others_max,
                                income_requirement_individual, income_requirement_combined)
        model_cred_companion_new.optimize()
        for a in range(1,len(i)+1):
            if Y[str(a)].x != Y_new[str(a)].x:
                diff = 1
        if diff == 1:
            new_max = delta
            break
    if diff == 1:
        travel_statement = "A better solution exists if the range of spending on travel is from " + str(user_travel_min) + " to " + str(new_max)
    else:
        travel_statement = "The current solution is optimal for any value spent on travel in the current selected range"
else:
    travel_statement = "The current solution is optimal for any value spent on travel in the current selected range"

In [16]:
#Define the range for groceries
if user_grocery_min != user_grocery_max:
    groceries_spending_delta = int((user_grocery_max - user_grocery_min)*5/100)
    diff = 0
    new_max = 0
    for delta in range(user_grocery_max,user_grocery_min,-int(groceries_spending_delta)):
        model_cred_companion_new,X_new,Y_new,Z_new,Y_APR_cash_new,Y_APR_purchase_new,X_APR_purchase_new = credcompanion_calculate('Cred_Companion_Groceries_' + str(delta),i,j,k,C,bonus_without_cond,bonus_with_cond,apr_cash_advance,apr_purchases,
                                bonus_rewards,annual_fees,
                                user_num_cards,user_loans,user_loans_amount,user_loans_months,user_payments,user_payments_amount,user_payments_months,
                                user_grocery_min,delta,
                                user_restaurant_min,user_restaurant_max,
                                user_bills_min,user_bills_max,
                                user_fuel_min,user_fuel_max,
                                user_travel_min, user_travel_max,
                                user_drugs_min, user_drugs_max,
                                user_others_min, user_others_max,
                                income_requirement_individual, income_requirement_combined)
        model_cred_companion_new.optimize()
        for a in range(1,len(i)+1):
            if Y[str(a)].x != Y_new[str(a)].x:
                diff = 1
        if diff == 1:
            new_max = delta
            break
    if diff == 1:
        grocery_statement = "A better solution exists if the range of spending on groceries is from " + str(user_grocery_min) + " to " + str(new_max)
    else:
        grocery_statement = "The current solution is optimal for any value spent on groceries in the current selected range"
else:
    grocery_statement = "The current solution is optimal for any value spent on groceries in the current selected range"

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0xce0c9083
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 0.10s
Presolved: 44785 rows, 58733 columns, 142540 nonzeros
Presolved model has 29312 SOS constraint(s)
Variable types: 43968 continuous, 14765 integer (14765 binary)
Found


*    0     0               0    5487.3551409 5487.35514  0.00%     -    1s

Explored 1 nodes (544 simplex iterations) in 1.48 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 5487.36 4380.31 3489.86 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.200302001297e+01, best bound 5.487355140886e+03, gap 24839.0999%
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0x8a3e01f5
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [

Variable types: 43968 continuous, 14765 integer (14765 binary)
Found heuristic solution: objective 3405.8560604
Found heuristic solution: objective 4321.5094736

Root relaxation: objective -5.421191e+03, 506 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    5421.1911510 5421.19115  0.00%     -    1s

Explored 1 nodes (506 simplex iterations) in 1.67 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 5421.19 4321.51 3405.86 

Optimal solution found (tolerance 1.00e-04)
Best objective -4.416096988641e+01, best bound 5.421191150987e+03, gap 12375.9785%
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0x6a62a8f0
Model has 14332 quadratic objectiv

  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 0.10s
Presolved: 44785 rows, 58733 columns, 142540 nonzeros
Presolved model has 29312 SOS constraint(s)
Variable types: 43968 continuous, 14765 integer (14765 binary)
Found heuristic solution: objective 3321.8560604
Found heuristic solution: objective 4262.7094736

Root relaxation: objective -5.355027e+03, 519 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    5355.0271611 5355.02716  0.00%     -    1s

Explored 1 nodes (519 simplex iterations) in 1.45 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 5355.03 4262.71 3321.86 

Optimal solution found (tolerance 1.00e-04)
Best o

Model fingerprint: 0xa9b9e22c
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 0.09s
Presolved: 44785 rows, 58733 columns, 142540 nonzeros
Presolved model has 29312 SOS constraint(s)
Variable types: 43968 continuous, 14765 integer (14765 binary)
Found heuristic solution: objective 3237.8560604
Found heuristic solution: objective 4203.9094736

Root relaxation: objective -5.288863e+03, 499 iterations, 0.01 seconds

    Nodes    |    Current Node  

In [17]:
#Define the range for restaurants
if user_restaurant_min != user_restaurant_max:
    restaurant_spending_delta = int((user_restaurant_max - user_restaurant_min)*5/100)
    diff = 0
    new_max = 0
    for delta in range(user_restaurant_max,user_restaurant_min,-int(restaurant_spending_delta)):
        model_cred_companion_new,X_new,Y_new,Z_new,Y_APR_cash_new,Y_APR_purchase_new,X_APR_purchase_new = credcompanion_calculate('Cred_Companion_restaurant_' + str(delta),i,j,k,C,bonus_without_cond,bonus_with_cond,apr_cash_advance,apr_purchases,
                                bonus_rewards,annual_fees,
                                user_num_cards,user_loans,user_loans_amount,user_loans_months,user_payments,user_payments_amount,user_payments_months,
                                user_grocery_min,user_grocery_max,
                                user_restaurant_min,delta,
                                user_bills_min,user_bills_max,
                                user_fuel_min,user_fuel_max,
                                user_travel_min, user_travel_max,
                                user_drugs_min, user_drugs_max,
                                user_others_min, user_others_max,
                                income_requirement_individual, income_requirement_combined)
        model_cred_companion_new.optimize()
        for a in range(1,len(i)+1):
            if Y[str(a)].x != Y_new[str(a)].x:
                diff = 1
        if diff == 1:
            new_max = delta
            break
    if diff == 1:
        restaurant_statement = "A better solution exists if the range of spending on restaurant is from " + str(user_restaurant_min) + " to " + str(new_max)
    else:
        restaurant_statement = "The current solution is optimal for any value spent on restaurant in the current selected range"
else:
    restaurant_statement = "The current solution is optimal for any value spent on restaurant in the current selected range"

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0xce0c9083
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 0.13s
Presolved: 44785 rows, 58733 columns, 142540 nonzeros
Presolved model has 29312 SOS constraint(s)
Variable types: 43968 continuous, 14765 integer (14765 binary)
Found


*    0     0               0    5521.0863328 5521.08633  0.00%     -    2s

Explored 1 nodes (518 simplex iterations) in 2.75 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 5521.09 4415.83 3537.86 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.573421193248e+01, best bound 5.521086332806e+03, gap 9806.0992%
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0x61913f16
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [1

Variable types: 43968 continuous, 14765 integer (14765 binary)
Found heuristic solution: objective 3513.8560604
Found heuristic solution: objective 4401.4294736

Root relaxation: objective -5.497086e+03, 528 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    5497.0863328 5497.08633  0.00%     -    3s

Explored 1 nodes (528 simplex iterations) in 3.28 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 5497.09 4401.43 3513.86 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.173421193248e+01, best bound 5.497086332806e+03, gap 17222.2714%
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0x983acfda
Model has 14332 quadratic objective

  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 0.10s
Presolved: 44785 rows, 58733 columns, 142540 nonzeros
Presolved model has 29312 SOS constraint(s)
Variable types: 43968 continuous, 14765 integer (14765 binary)
Found heuristic solution: objective 3489.8560604
Found heuristic solution: objective 4387.0294736

Root relaxation: objective -5.473086e+03, 536 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    5473.0863328 5473.08633  0.00%     -    1s

Explored 1 nodes (536 simplex iterations) in 1.79 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 5473.09 4387.03 3489.86 

Optimal solution found (tolerance 1.00e-04)
Best o

Model fingerprint: 0x6edf8dd1
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 0.25s
Presolved: 44785 rows, 58733 columns, 142540 nonzeros
Presolved model has 29312 SOS constraint(s)
Variable types: 43968 continuous, 14765 integer (14765 binary)
Found heuristic solution: objective 3465.8560604
Found heuristic solution: objective 4372.6294736

Root relaxation: objective -5.449086e+03, 519 iterations, 0.01 seconds

    Nodes    |    Current Node  


Solution count 3: 5429.89 4361.11 3446.66 

Optimal solution found (tolerance 1.00e-04)
Best objective -3.546578806752e+01, best bound 5.429886332806e+03, gap 15410.2092%
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0x8436344e
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 

In [18]:
#Define the range for fuel
if user_fuel_min != user_fuel_max:
    fuel_spending_delta = int((user_fuel_max - user_fuel_min)*5/100)
    diff = 0
    new_max = 0
    for delta in range(user_fuel_max,user_fuel_min,-int(fuel_spending_delta)):
        model_cred_companion_new,X_new,Y_new,Z_new,Y_APR_cash_new,Y_APR_purchase_new,X_APR_purchase_new = credcompanion_calculate('Cred_Companion_fuel_' + str(delta),i,j,k,C,bonus_without_cond,bonus_with_cond,apr_cash_advance,apr_purchases,
                                bonus_rewards,annual_fees,
                                user_num_cards,user_loans,user_loans_amount,user_loans_months,user_payments,user_payments_amount,user_payments_months,
                                user_grocery_min,user_grocery_max,
                                user_restaurant_min,user_restaurant_max,
                                user_bills_min,user_bills_max,
                                user_fuel_min,delta,
                                user_travel_min, user_travel_max,
                                user_drugs_min, user_drugs_max,
                                user_others_min, user_others_max,
                                income_requirement_individual, income_requirement_combined)
        model_cred_companion_new.optimize()
        for a in range(1,len(i)+1):
            if Y[str(a)].x != Y_new[str(a)].x:
                diff = 1
        if diff == 1:
            new_max = delta
            break
    if diff == 1:
        fuel_statement = "A better solution exists if the range of spending on fuel is from " + str(user_fuel_min) + " to " + str(new_max)
    else:
        fuel_statement = "The current solution is optimal for any value spent on fuel in the current selected range"
else:
    fuel_statement = "The current solution is optimal for any value spent on fuel in the current selected range"

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0xce0c9083
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 0.10s
Presolved: 44785 rows, 58733 columns, 142540 nonzeros
Presolved model has 29312 SOS constraint(s)
Variable types: 43968 continuous, 14765 integer (14765 binary)
Found


*    0     0               0    5526.9618913 5526.96189  0.00%     -    1s

Explored 1 nodes (543 simplex iterations) in 1.72 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 5526.96 4415.83 3545.54 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.160977046934e+01, best bound 5.526961891343e+03, gap 8870.9178%
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0xbb846f15
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [1

Variable types: 43968 continuous, 14765 integer (14765 binary)
Found heuristic solution: objective 3531.1360604
Found heuristic solution: objective 4401.4294736

Root relaxation: objective -5.510306e+03, 523 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    5510.3063395 5510.30634  0.00%     -    1s

Explored 1 nodes (523 simplex iterations) in 1.54 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 5510.31 4401.43 3531.14 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.495421864042e+01, best bound 5.510306339514e+03, gap 12157.5956%
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0x127eeb41
Model has 14332 quadratic objective

  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 0.10s
Presolved: 44785 rows, 58733 columns, 142540 nonzeros
Presolved model has 29312 SOS constraint(s)
Variable types: 43968 continuous, 14765 integer (14765 binary)
Found heuristic solution: objective 3516.7360604
Found heuristic solution: objective 4387.0294736

Root relaxation: objective -5.493651e+03, 522 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    5493.6507877 5493.65079  0.00%     -    1s

Explored 1 nodes (522 simplex iterations) in 1.56 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 5493.65 4387.03 3516.74 

Optimal solution found (tolerance 1.00e-04)
Best o

Model fingerprint: 0xc7f9d512
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 0.10s
Presolved: 44785 rows, 58733 columns, 142540 nonzeros
Presolved model has 29312 SOS constraint(s)
Variable types: 43968 continuous, 14765 integer (14765 binary)
Found heuristic solution: objective 3502.3360604
Found heuristic solution: objective 4372.6294736

Root relaxation: objective -5.476995e+03, 524 iterations, 0.01 seconds

    Nodes    |    Current Node  


Solution count 3: 5463.67 4361.11 3490.82 

Optimal solution found (tolerance 1.00e-04)
Best objective -1.681326480557e+00, best bound 5.463670794393e+03, gap 325061.9189%
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0x6baa1b13
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time:

In [19]:
#Define the range for drugs
if user_drugs_min != user_drugs_max:
    drugs_spending_delta = int((user_drugs_max - user_drugs_min)*5/100)
    diff = 0
    new_max = 0
    for delta in range(user_drugs_max,user_drugs_min,-int(drugs_spending_delta)):
        model_cred_companion_new,X_new,Y_new,Z_new,Y_APR_cash_new,Y_APR_purchase_new,X_APR_purchase_new = credcompanion_calculate('Cred_Companion_drugs_' + str(delta),i,j,k,C,bonus_without_cond,bonus_with_cond,apr_cash_advance,apr_purchases,
                                bonus_rewards,annual_fees,
                                user_num_cards,user_loans,user_loans_amount,user_loans_months,user_payments,user_payments_amount,user_payments_months,
                                user_grocery_min,user_grocery_max,
                                user_restaurant_min,user_restaurant_max,
                                user_bills_min,user_bills_max,
                                user_fuel_min,user_fuel_max,
                                user_travel_min, user_travel_max,
                                user_drugs_min, delta,
                                user_others_min, user_others_max,
                                income_requirement_individual, income_requirement_combined)
        model_cred_companion_new.optimize()
        for a in range(1,len(i)+1):
            if Y[str(a)].x != Y_new[str(a)].x:
                diff = 1
        if diff == 1:
            new_max = delta
            break
    if diff == 1:
        drugs_statement = "A better solution exists if the range of spending on drugs is from " + str(user_drugs_min) + " to " + str(new_max)
    else:
        drugs_statement = "The current solution is optimal for any value spent on drugs in the current selected range"
else:
    drugs_statement = "The current solution is optimal for any value spent on drugs in the current selected range"

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0xce0c9083
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 0.11s
Presolved: 44785 rows, 58733 columns, 142540 nonzeros
Presolved model has 29312 SOS constraint(s)
Variable types: 43968 continuous, 14765 integer (14765 binary)
Found


*    0     0               0    5492.2863328 5492.28633  0.00%     -    1s

Explored 1 nodes (520 simplex iterations) in 1.41 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 5492.29 4398.55 3542.66 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.693421193248e+01, best bound 5.492286332806e+03, gap 20291.4870%
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0xb4f20ef1
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [

Variable types: 43968 continuous, 14765 integer (14765 binary)
Found heuristic solution: objective 3324.6560604
Found heuristic solution: objective 4362.5494736

Root relaxation: objective -5.432286e+03, 544 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    5432.2863328 5432.28633  0.00%     -    1s

Explored 1 nodes (544 simplex iterations) in 1.51 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 5432.29 4362.55 3324.66 

Optimal solution found (tolerance 1.00e-04)
Best objective -3.306578806752e+01, best bound 5.432286332806e+03, gap 16528.7218%
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0xa079a4ce
Model has 14332 quadratic objectiv

  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 0.09s
Presolved: 44785 rows, 58733 columns, 142540 nonzeros
Presolved model has 29312 SOS constraint(s)
Variable types: 43968 continuous, 14765 integer (14765 binary)
Found heuristic solution: objective 3306.6560604
Found heuristic solution: objective 4326.5494736

Root relaxation: objective -5.372286e+03, 557 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    5372.2863328 5372.28633  0.00%     -    1s

Explored 1 nodes (557 simplex iterations) in 1.44 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 5372.29 4326.55 3306.66 

Optimal solution found (tolerance 1.00e-04)
Best o

Model fingerprint: 0xca1ee763
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 0.10s
Presolved: 44785 rows, 58733 columns, 142540 nonzeros
Presolved model has 29312 SOS constraint(s)
Variable types: 43968 continuous, 14765 integer (14765 binary)
Found heuristic solution: objective 3288.6560604
Found heuristic solution: objective 4290.5494736

Root relaxation: objective -5.312286e+03, 553 iterations, 0.01 seconds

    Nodes    |    Current Node  

In [20]:
#Define the range for bills
if user_bills_min != user_bills_max:
    bills_spending_delta = int((user_bills_max - user_bills_min)*5/100)
    diff = 0
    new_max = 0
    for delta in range(user_bills_max,user_bills_min,-int(bills_spending_delta)):
        model_cred_companion_new,X_new,Y_new,Z_new,Y_APR_cash_new,Y_APR_purchase_new,X_APR_purchase_new = credcompanion_calculate('Cred_Companion_bills_' + str(delta),i,j,k,C,bonus_without_cond,bonus_with_cond,apr_cash_advance,apr_purchases,
                                bonus_rewards,annual_fees,
                                user_num_cards,user_loans,user_loans_amount,user_loans_months,user_payments,user_payments_amount,user_payments_months,
                                user_grocery_min,user_grocery_max,
                                user_restaurant_min,user_restaurant_max,
                                user_bills_min,delta,
                                user_fuel_min,user_fuel_max,
                                user_travel_min, user_travel_max,
                                user_drugs_min, user_drugs_max,
                                user_others_min, user_others_max,
                                income_requirement_individual, income_requirement_combined)
        model_cred_companion_new.optimize()
        for a in range(1,len(i)+1):
            if Y[str(a)].x != Y_new[str(a)].x:
                diff = 1
        if diff == 1:
            new_max = delta
            break
    if diff == 1:
        bills_statement = "A better solution exists if the range of spending on bills is from " + str(user_bills_min) + " to " + str(new_max)
    else:
        bills_statement = "The current solution is optimal for any value spent on bills in the current selected range"
else:
    bills_statement = "The current solution is optimal for any value spent on bills in the current selected range"

In [21]:
#Define the range for other categories
if user_others_min != user_others_max:
    others_spending_delta = int((user_others_max - user_others_min)*5/100)
    diff = 0
    new_max = 0
    for delta in range(user_others_max,user_others_min,-int(others_spending_delta)):
        model_cred_companion_new,X_new,Y_new,Z_new,Y_APR_cash_new,Y_APR_purchase_new,X_APR_purchase_new = credcompanion_calculate('Cred_Companion_bills_' + str(delta),i,j,k,C,bonus_without_cond,bonus_with_cond,apr_cash_advance,apr_purchases,
                                bonus_rewards,annual_fees,
                                user_num_cards,user_loans,user_loans_amount,user_loans_months,user_payments,user_payments_amount,user_payments_months,
                                user_grocery_min,user_grocery_max,
                                user_restaurant_min,user_restaurant_max,
                                user_bills_min,user_bills_max,
                                user_fuel_min,user_fuel_max,
                                user_travel_min, user_travel_max,
                                user_drugs_min, user_drugs_max,
                                user_others_min, delta,
                                income_requirement_individual, income_requirement_combined)
        model_cred_companion_new.optimize()
        for a in range(1,len(i)+1):
            if Y[str(a)].x != Y_new[str(a)].x:
                diff = 1
        if diff == 1:
            new_max = delta
            break
    if diff == 1:
        others_statement = "A better solution exists if the range of spending on other categories is from " + str(user_others_min) + " to " + str(new_max)
    else:
        others_statement = "The current solution is optimal for any value spent on other categories in the current selected range"
else:
    others_statement = "The current solution is optimal for any value spent on other categories in the current selected range"

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0xce0c9083
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 0.09s
Presolved: 44785 rows, 58733 columns, 142540 nonzeros
Presolved model has 29312 SOS constraint(s)
Variable types: 43968 continuous, 14765 integer (14765 binary)
Found


*    0     0               0    5530.6863328 5530.68633  0.00%     -    1s

Explored 1 nodes (512 simplex iterations) in 1.45 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 5530.69 4421.59 3554.18 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.533421193248e+01, best bound 5.530686332806e+03, gap 8365.2224%
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0xb16f180d
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [1

Variable types: 43968 continuous, 14765 integer (14765 binary)
Found heuristic solution: objective 3550.5760604
Found heuristic solution: objective 4414.3894736

Root relaxation: objective -5.518686e+03, 518 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    5518.6863328 5518.68633  0.00%     -    1s

Explored 1 nodes (518 simplex iterations) in 1.42 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 5518.69 4414.39 3550.58 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.333421193248e+01, best bound 5.518686332806e+03, gap 10247.3664%
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0xa8fbc991
Model has 14332 quadratic objective

  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 0.09s
Presolved: 44785 rows, 58733 columns, 142540 nonzeros
Presolved model has 29312 SOS constraint(s)
Variable types: 43968 continuous, 14765 integer (14765 binary)
Found heuristic solution: objective 3546.9760604
Found heuristic solution: objective 4407.1894736

Root relaxation: objective -5.506686e+03, 529 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    5506.6863328 5506.68633  0.00%     -    1s

Explored 1 nodes (529 simplex iterations) in 1.38 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 5506.69 4407.19 3546.98 

Optimal solution found (tolerance 1.00e-04)
Best o

Model fingerprint: 0x61ca84e3
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 0.21s
Presolved: 44785 rows, 58733 columns, 142540 nonzeros
Presolved model has 29312 SOS constraint(s)
Variable types: 43968 continuous, 14765 integer (14765 binary)
Found heuristic solution: objective 3543.3760604
Found heuristic solution: objective 4399.9894736

Root relaxation: objective -5.494686e+03, 524 iterations, 0.01 seconds

    Nodes    |    Current Node  

In [22]:
#Define the range for user loans
if user_loans == 'Yes':
    loans_delta = int((user_loans_amount)*5/100)
    diff = 0
    new_max = 0
    for delta in range(user_loans_amount,0,-int(loans_delta)):
        model_cred_companion_new,X_new,Y_new,Z_new,Y_APR_cash_new,Y_APR_purchase_new,X_APR_purchase_new = credcompanion_calculate('Cred_Companion_bills_' + str(delta),i,j,k,C,bonus_without_cond,bonus_with_cond,apr_cash_advance,apr_purchases,
                                bonus_rewards,annual_fees,
                                user_num_cards,user_loans,delta,user_loans_months,user_payments,user_payments_amount,user_payments_months,
                                user_grocery_min,user_grocery_max,
                                user_restaurant_min,user_restaurant_max,
                                user_bills_min,user_bills_max,
                                user_fuel_min,user_fuel_max,
                                user_travel_min, user_travel_max,
                                user_drugs_min, user_drugs_max,
                                user_others_min, user_others_max,
                                income_requirement_individual, income_requirement_combined)
        model_cred_companion_new.optimize()
        for a in range(1,len(i)+1):
            if Y[str(a)].x != Y_new[str(a)].x:
                diff = 1
        if diff == 1:
            new_max = delta
            break
    if diff == 1:
        loans_statement = "A better solution exists if the cash advance amount is less than " + str(new_max)
    else:
        loans_statement = "The current solution is optimal for any value of cash advance taken upto " + str(user_loans_amount)
else:
    loans_statement = "NA"

In [23]:
#Define the range for user payments
if user_payments == 'No':
    payments_delta = int((user_payments_amount)*5/100)
    diff = 0
    new_max = 0
    for delta in range(user_payments_amount,0-int(payments_delta),-int(payments_delta)):
        model_cred_companion_new,X_new,Y_new,Z_new,Y_APR_cash_new,Y_APR_purchase_new,X_APR_purchase_new = credcompanion_calculate('Cred_Companion_bills_' + str(delta),i,j,k,C,bonus_without_cond,bonus_with_cond,apr_cash_advance,apr_purchases,
                                bonus_rewards,annual_fees,
                                user_num_cards,user_loans,user_loans_amount,user_loans_months,user_payments,delta,user_payments_months,
                                user_grocery_min,user_grocery_max,
                                user_restaurant_min,user_restaurant_max,
                                user_bills_min,user_bills_max,
                                user_fuel_min,user_fuel_max,
                                user_travel_min, user_travel_max,
                                user_drugs_min, user_drugs_max,
                                user_others_min, user_others_max,
                                income_requirement_individual, income_requirement_combined)
        model_cred_companion_new.optimize()
        for a in range(1,len(i)+1):
            if Y[str(a)].x != Y_new[str(a)].x:
                diff = 1
        if diff == 1:
            new_max = delta
            break
    if diff == 1:
        payments_statement = "A better solution exists if the credit repayment shortfall is less than " + str(new_max)
    else:
        payments_statement = "The current solution is optimal for any value of credit repayment shortfalls upto " + str(user_loans_amount)
else:
    payments_statement = "NA"

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0xce0c9083
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 0.11s
Presolved: 44785 rows, 58733 columns, 142540 nonzeros
Presolved model has 29312 SOS constraint(s)
Variable types: 43968 continuous, 14765 integer (14765 binary)
Found


*    0     0               0    4952.0276986 4952.02770  0.00%     -    2s

Explored 1 nodes (537 simplex iterations) in 2.61 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 4952.03 3804.02 3010.52 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.797460019456e+02, best bound 4.952027698644e+03, gap 754.1719%
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17889 rows, 17941 columns and 35746 nonzeros
Model fingerprint: 0xcf821f3e
Model has 14332 quadratic objective terms
Model has 894 quadratic constraints
Variable types: 17680 continuous, 261 integer (261 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+09]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+04]
  Objective range  [3e+01, 2e+03]
  QObjective range [7e-03, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [1e

  QRHS range       [1e+02, 6e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 0.11s
Presolved: 44785 rows, 58733 columns, 142540 nonzeros
Presolved model has 29312 SOS constraint(s)
Variable types: 43968 continuous, 14765 integer (14765 binary)
Found heuristic solution: objective 2327.3518332
Found heuristic solution: objective 3024.8676550

Root relaxation: objective -4.185539e+03, 410 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4185.53888    0    3 3024.86765 4185.53888  38.4%     -    2s
H    0     0                    4151.9697179 4185.53888  0.81%     -    2s
     0     0 4163.91525    0    2 4151.96972 4163.91525  0.29%     -    2s
     0     0     cutoff    0      4151.96972 4151.96972  0.00%     -    2s

Explored 

  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  QRHS range       [1e+02, 4e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3219 rows and 3183 columns
Presolve time: 0.25s
Presolved: 44785 rows, 58733 columns, 142540 nonzeros
Presolved model has 29312 SOS constraint(s)
Variable types: 43968 continuous, 14765 integer (14765 binary)
Found heuristic solution: objective 1780.8166212
Found heuristic solution: objective 2375.5024022

Root relaxation: objective -3.540616e+03, 407 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3540.61565    0    3 2375.50240 3540.61565  49.0%     -    1s
H    0     0                    3421.8191923 3540.61565  3.47%     -    1s
     0     0 3524.51096    0    6 3421.81919 3524.51096  3.00%     -    1s
     0     0 3524

<a id='store_output'></a>
## Code Block - Store output in excel
[Back to table of contents](#contents)

In [24]:
credcompanion_output(file_path,model_cred_companion,i,j,k,X,Y,C,Z,bonus_without_cond,bonus_with_cond,user_num_years,
                         annual_fees,user_loans,user_payments,user_loans_amount,apr_cash_advance,user_loans_months,Y_APR_cash,
                        Y_APR_purchase,X_APR_purchase,apr_purchases,user_payments_months,
                     travel_statement,grocery_statement,restaurant_statement,fuel_statement,drugs_statement,
                     bills_statement,others_statement,loans_statement,payments_statement)